# Setup Libraries and Dependencies

In [14]:
!python --version

Python 3.11.11


## Pip installation

In [15]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

## AWS CLI and AWS Python SDK (boto3) Installation

In [16]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

## SageMaker Installation

In [17]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

## PyAthena Installation

In [18]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

## AWS Data Wrangler

In [19]:
!pip install --disable-pip-version-check -q awswrangler==3.11.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.20.0 requires botocore<1.36.24,>=1.36.20, but you have botocore 1.37.24 which is incompatible.
amazon-sagemaker-sql-execution 0.1.6 requires pyathena<4,>=3.3.0, but you have pyathena 2.1.0 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-core 1.2 requires matplotlib<3.11,>=3.7.0, but you have matplotlib 3.1.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 wh

## Matplotlib Installation

In [20]:
!pip install --disable-pip-version-check -q matplotlib==3.1.3

## Seaborn Installation

In [21]:
!pip install --disable-pip-version-check -q seaborn==0.13.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.2 requires omegaconf<2.3.0,>=2.1.1, but you have omegaconf 2.3.0 which is incompatible.
mlflow 2.20.3 requires sqlalchemy<3,>=1.4.0, but you have sqlalchemy 1.3.22 which is incompatible.


## Workcloud installation

In [22]:
!pip install wordcloud

## Summarize

In [23]:
setup_dependencies_passed = True

In [24]:
%store setup_dependencies_passed

Stored 'setup_dependencies_passed' (bool)


In [25]:
%store

Stored variables and their in-db values:
s3_private_path                         -> 's3://sagemaker-us-east-1-445732154141/linkedin_da
s3_public_path                          -> 's3://linkedin-postings'
setup_dependencies_passed               -> True
setup_iam_roles_passed                  -> True
setup_instance_check_passed             -> True
setup_s3_bucket_passed                  -> True


## Check environment was created correctly

In [26]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name="ec2", region_name=region)
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

AttributeError: module 'urllib3.util' has no attribute 'PROTOCOL_TLS'

### _Note: This workshop requires SageMaker Studio and will not work properly in classic SageMaker Notebooks._

In [ ]:
import json

notebook_instance_name = None

try:
    with open("/opt/ml/metadata/resource-metadata.json") as notebook_info:
        data = json.load(notebook_info)
        domain_id = data["DomainId"]
        resource_arn = data["ResourceArn"]
        region = resource_arn.split(":")[3]
        name = data["ResourceName"]
    print("DomainId: {}".format(domain_id))
    print("Name: {}".format(name))
except:
    print("+++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: COULD NOT RETRIEVE THE METADATA.")
    print("+++++++++++++++++++++++++++++++++++++++++")

In [ ]:
describe_domain_response = sm.describe_domain(DomainId=domain_id)
print(describe_domain_response["Status"])

In [ ]:
try:
    get_status_response = sm.get_sagemaker_servicecatalog_portfolio_status()
    print(get_status_response["Status"])
except:
    pass

## Summary: Check All Required Settings Are Set Correctly

In [ ]:
if (
    describe_domain_response["Status"] == "InService"
    and get_status_response["Status"] == "Enabled"
    and "default" in name
):
    setup_instance_check_passed = True
    print("[OK] Checks passed!  Great Job!!  Please Continue.")
else:
    setup_instance_check_passed = False
    print("+++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR]: WE HAVE IDENTIFIED A MISCONFIGURATION.")
    print(describe_domain_response["Status"])
    print(get_status_response["Status"])
    print(name)
    print("+++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(setup_instance_check_passed)

In [ ]:
%store setup_instance_check_passed

In [ ]:
%store

## Create S3 Bucket

In [ ]:
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:
setup_s3_bucket_passed = False

In [ ]:
print("Default bucket: {}".format(bucket))

# Verify s3_bucket Bucket Creation

In [ ]:
%%bash

aws s3 ls s3://${bucket}/

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

In [ ]:
%store setup_s3_bucket_passed

In [ ]:
%store

## Update IAM Roles and Policies

In [ ]:
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

### Get SageMaker Execution Role Name

In [ ]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

In [ ]:
setup_iam_roles_passed = False

## Pre-Requisite: SageMaker notebook instance ExecutionRole contains IAMFullAccess Policy.

In [ ]:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        break

setup_iam_roles_passed = True
print("[OK] You are all set up to continue with this workshop!")

In [ ]:
%store setup_iam_roles_passed

In [ ]:
%store

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}